In [ ]:
from transformers import OneFormerProcessor, OneFormerForUniversalSegmentation
from PIL import Image
import requests
import matplotlib.pyplot as plt
import os


# load image

In [ ]:
image_name = "../../data/ade20k.jpeg"
# read image if the file exists, else read from url.
if os.path.exists(image_name):
  image = Image.open(image_name)
else:
  url = "https://huggingface.co/datasets/shi-labs/oneformer_demo/resolve/main/ade20k.jpeg"
  response = requests.get(url, stream=True)
  response.raise_for_status()  # Check for HTTP errors
  image = Image.open(response.raw)

In [ ]:
image

# run segmentation

In [ ]:
model_name = "shi-labs/oneformer_coco_swin_large"
task_type = "semantic"

In [ ]:
processor = OneFormerProcessor.from_pretrained(
model_name
)  # Load once here
model = OneFormerForUniversalSegmentation.from_pretrained(
model_name
)

In [ ]:
inputs = processor(images=image, task_inputs=[
                    "semantic"], return_tensors="pt")


In [ ]:
model

In [ ]:
import torch

In [ ]:
with torch.no_grad():
  outputs = model(**inputs)


In [ ]:
predicted_map = processor.post_process_semantic_segmentation(
    outputs, target_sizes=[image.size[::-1]]
)[0]

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Original Image")
plt.axis("off")
plt.subplot(1, 2, 2)
plt.imshow(predicted_map)
plt.title("Segmentation")
plt.axis("off")
plt.show()
plt.savefig("../../result/oneformer_segm.png")